In [1]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout, Add, LSTM, GRU, RNN, Layer
from keras import backend as K
from keras.optimizers import Adam,SGD
import tensorflow as tf
from keras import Model, regularizers, activations, initializers
from keras.constraints import Constraint
import pickle

# load and save mat file
import h5py
import hdf5storage

In [2]:
with h5py.File('test_collected_data.mat', 'r') as file:
    U = file['U'][:]
    U = np.array(U)

    Y = file['Y'][:]
    Y = np.array(Y)

In [3]:
# get X and y data for training and testing

U_input = list()
Y_output = list()

for i in range(U.shape[0]-1):
    if i % 5 == 0:
        U_input.append(U[i:i+5, :])
        Y_output.append(Y[i:i+5, :])

In [4]:
U_input = np.array(U_input)
print(U_input.shape)
Y_output = np.array(Y_output)
print(Y_output.shape)

(4000, 5, 2)
(4000, 5, 2)


In [5]:
RNN_input = U_input
print("RNN_input shape is {}".format(RNN_input.shape))

RNN_input shape is (4000, 5, 2)


In [6]:
RNN_output = Y_output
print("RNN_output shape is {}".format(RNN_output.shape))

RNN_output shape is (4000, 5, 2)


In [7]:
# set the seed for reproducibility
tf.random.set_seed(42)

# define parameter values
num_step = 5
num_dims = 2

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(RNN_input, RNN_output, test_size=0.1, random_state=123)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# define scalers for both X and y base on training data only
scaler_X = preprocessing.MinMaxScaler().fit(X_train.reshape(-1, num_dims))
scaler_y = preprocessing.MinMaxScaler().fit(y_train.reshape(-1, 2))

X_train = scaler_X.transform(X_train.reshape(-1, num_dims)).reshape(-1,num_step,num_dims)
X_test = scaler_X.transform(X_test.reshape(-1, num_dims)).reshape(-1,num_step,num_dims)
y_train = scaler_y.transform(y_train.reshape(-1,2)).reshape(-1,num_step,2)
y_test_normalized = scaler_y.transform(y_test.reshape(-1,2)).reshape(-1,num_step,2)

# LSTM model training and evaluation
nh = 64
input = Input(shape=(X_train.shape[1],X_train.shape[2]))

x = LSTM(nh,
      return_sequences=True,
      )(input)
x = LSTM(nh,
      return_sequences=True,
      )(x)
x = Dense(2,
        activation='linear',
        )(x)

model = Model(input, x)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanSquaredError()])
history = model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_split=0, verbose=2)

loss = model.evaluate(X_test, y_test_normalized, batch_size=256)
test_loss = loss[0]
print(test_loss)

model.summary()

(3600, 5, 2)
(400, 5, 2)
(3600, 5, 2)
(400, 5, 2)
Epoch 1/1000
15/15 - 3s - loss: 0.1415 - mean_squared_error: 0.1415 - 3s/epoch - 183ms/step
Epoch 2/1000
15/15 - 0s - loss: 0.0797 - mean_squared_error: 0.0797 - 183ms/epoch - 12ms/step
Epoch 3/1000
15/15 - 0s - loss: 0.0658 - mean_squared_error: 0.0658 - 181ms/epoch - 12ms/step
Epoch 4/1000
15/15 - 0s - loss: 0.0563 - mean_squared_error: 0.0563 - 353ms/epoch - 24ms/step
Epoch 5/1000
15/15 - 0s - loss: 0.0483 - mean_squared_error: 0.0483 - 165ms/epoch - 11ms/step
Epoch 6/1000
15/15 - 0s - loss: 0.0417 - mean_squared_error: 0.0417 - 262ms/epoch - 17ms/step
Epoch 7/1000
15/15 - 0s - loss: 0.0373 - mean_squared_error: 0.0373 - 284ms/epoch - 19ms/step
Epoch 8/1000
15/15 - 0s - loss: 0.0337 - mean_squared_error: 0.0337 - 306ms/epoch - 20ms/step
Epoch 9/1000
15/15 - 0s - loss: 0.0298 - mean_squared_error: 0.0298 - 381ms/epoch - 25ms/step
Epoch 10/1000
15/15 - 0s - loss: 0.0255 - mean_squared_error: 0.0255 - 246ms/epoch - 16ms/step
Epoch 11/10

Epoch 87/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 352ms/epoch - 23ms/step
Epoch 88/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 205ms/epoch - 14ms/step
Epoch 89/1000
15/15 - 0s - loss: 0.0061 - mean_squared_error: 0.0061 - 467ms/epoch - 31ms/step
Epoch 90/1000
15/15 - 0s - loss: 0.0061 - mean_squared_error: 0.0061 - 257ms/epoch - 17ms/step
Epoch 91/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 177ms/epoch - 12ms/step
Epoch 92/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 167ms/epoch - 11ms/step
Epoch 93/1000
15/15 - 0s - loss: 0.0061 - mean_squared_error: 0.0061 - 238ms/epoch - 16ms/step
Epoch 94/1000
15/15 - 0s - loss: 0.0061 - mean_squared_error: 0.0061 - 194ms/epoch - 13ms/step
Epoch 95/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 150ms/epoch - 10ms/step
Epoch 96/1000
15/15 - 0s - loss: 0.0061 - mean_squared_error: 0.0061 - 147ms/epoch - 10ms/step
Epoch 97/1000
15/15 - 0s - loss: 0.0061 - mean_squ

Epoch 173/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 159ms/epoch - 11ms/step
Epoch 174/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 226ms/epoch - 15ms/step
Epoch 175/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 387ms/epoch - 26ms/step
Epoch 176/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 371ms/epoch - 25ms/step
Epoch 177/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 195ms/epoch - 13ms/step
Epoch 178/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 314ms/epoch - 21ms/step
Epoch 179/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 330ms/epoch - 22ms/step
Epoch 180/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 365ms/epoch - 24ms/step
Epoch 181/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 446ms/epoch - 30ms/step
Epoch 182/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 202ms/epoch - 13ms/step
Epoch 183/1000
15/15 - 0s - loss: 0.0060

Epoch 259/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 141ms/epoch - 9ms/step
Epoch 260/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 173ms/epoch - 12ms/step
Epoch 261/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 282ms/epoch - 19ms/step
Epoch 262/1000
15/15 - 0s - loss: 0.0061 - mean_squared_error: 0.0061 - 157ms/epoch - 10ms/step
Epoch 263/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 250ms/epoch - 17ms/step
Epoch 264/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 469ms/epoch - 31ms/step
Epoch 265/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 325ms/epoch - 22ms/step
Epoch 266/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 469ms/epoch - 31ms/step
Epoch 267/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 279ms/epoch - 19ms/step
Epoch 268/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 277ms/epoch - 18ms/step
Epoch 269/1000
15/15 - 0s - loss: 0.0060 

Epoch 345/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 194ms/epoch - 13ms/step
Epoch 346/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 212ms/epoch - 14ms/step
Epoch 347/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 179ms/epoch - 12ms/step
Epoch 348/1000
15/15 - 1s - loss: 0.0060 - mean_squared_error: 0.0060 - 543ms/epoch - 36ms/step
Epoch 349/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 314ms/epoch - 21ms/step
Epoch 350/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 251ms/epoch - 17ms/step
Epoch 351/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 374ms/epoch - 25ms/step
Epoch 352/1000
15/15 - 1s - loss: 0.0060 - mean_squared_error: 0.0060 - 501ms/epoch - 33ms/step
Epoch 353/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 179ms/epoch - 12ms/step
Epoch 354/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 350ms/epoch - 23ms/step
Epoch 355/1000
15/15 - 1s - loss: 0.0060

...

Epoch 603/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 256ms/epoch - 17ms/step
Epoch 604/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 488ms/epoch - 33ms/step
Epoch 605/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 139ms/epoch - 9ms/step
Epoch 606/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 164ms/epoch - 11ms/step
Epoch 607/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 245ms/epoch - 16ms/step
Epoch 608/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 156ms/epoch - 10ms/step
Epoch 609/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 197ms/epoch - 13ms/step
Epoch 610/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 143ms/epoch - 10ms/step
Epoch 611/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 149ms/epoch - 10ms/step
Epoch 612/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 172ms/epoch - 11ms/step
Epoch 613/1000
15/15 - 0s - loss: 0.0060 

Epoch 689/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 428ms/epoch - 29ms/step
Epoch 690/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 310ms/epoch - 21ms/step
Epoch 691/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 441ms/epoch - 29ms/step
Epoch 692/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 465ms/epoch - 31ms/step
Epoch 693/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 326ms/epoch - 22ms/step
Epoch 694/1000
15/15 - 1s - loss: 0.0060 - mean_squared_error: 0.0060 - 520ms/epoch - 35ms/step
Epoch 695/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 345ms/epoch - 23ms/step
Epoch 696/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 444ms/epoch - 30ms/step
Epoch 697/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 372ms/epoch - 25ms/step
Epoch 698/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 398ms/epoch - 27ms/step
Epoch 699/1000
15/15 - 0s - loss: 0.0060

Epoch 775/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 300ms/epoch - 20ms/step
Epoch 776/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 268ms/epoch - 18ms/step
Epoch 777/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 376ms/epoch - 25ms/step
Epoch 778/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 400ms/epoch - 27ms/step
Epoch 779/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 447ms/epoch - 30ms/step
Epoch 780/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 357ms/epoch - 24ms/step
Epoch 781/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 335ms/epoch - 22ms/step
Epoch 782/1000
15/15 - 1s - loss: 0.0060 - mean_squared_error: 0.0060 - 507ms/epoch - 34ms/step
Epoch 783/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 422ms/epoch - 28ms/step
Epoch 784/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 307ms/epoch - 20ms/step
Epoch 785/1000
15/15 - 0s - loss: 0.0060

Epoch 861/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 308ms/epoch - 21ms/step
Epoch 862/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 224ms/epoch - 15ms/step
Epoch 863/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 372ms/epoch - 25ms/step
Epoch 864/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 367ms/epoch - 24ms/step
Epoch 865/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 477ms/epoch - 32ms/step
Epoch 866/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 344ms/epoch - 23ms/step
Epoch 867/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 363ms/epoch - 24ms/step
Epoch 868/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 339ms/epoch - 23ms/step
Epoch 869/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 213ms/epoch - 14ms/step
Epoch 870/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 346ms/epoch - 23ms/step
Epoch 871/1000
15/15 - 0s - loss: 0.0060

Epoch 947/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 187ms/epoch - 12ms/step
Epoch 948/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 262ms/epoch - 17ms/step
Epoch 949/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 197ms/epoch - 13ms/step
Epoch 950/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 189ms/epoch - 13ms/step
Epoch 951/1000
15/15 - 0s - loss: 0.0059 - mean_squared_error: 0.0059 - 210ms/epoch - 14ms/step
Epoch 952/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 171ms/epoch - 11ms/step
Epoch 953/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 422ms/epoch - 28ms/step
Epoch 954/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 452ms/epoch - 30ms/step
Epoch 955/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 422ms/epoch - 28ms/step
Epoch 956/1000
15/15 - 0s - loss: 0.0060 - mean_squared_error: 0.0060 - 391ms/epoch - 26ms/step
Epoch 957/1000
15/15 - 0s - loss: 0.0060

In [8]:
# save model and scalers
model.save('lstm_energy.keras')
pickle.dump(scaler_X, open('lstm_scaler_X', 'wb'))
pickle.dump(scaler_y, open('lstm_scaler_y', 'wb'))